## Causal Inference Exploration

In [1]:
import pandas as pd
import numpy as np

### **Apr 25 Session**
Checking on viability of socially engageed as a treatment. Not viable, entire dataset not socially engaged.

In [11]:
gstore["socialEngagementType"].value_counts()

Not Socially Engaged    903653
Name: socialEngagementType, dtype: int64

### **Apr 24 Session**

create a flag column for if something is a cpc session
Find the date of the first record for which medium = cpc
Find the date of the last session in which medium = cpc
Creating treatment group dataset. 
Session records for which it is the visitorid's first visit with medium = cpc
Then another one in which 

In [2]:
gstore = pd.read_csv("/Users/aoluwolerotimi/Datasets/train_dejsonified.csv")

/var/folders/z1/1zds15w97_g7_fhs4_83dc3m0000gn/T/ipykernel_56232/3136096564.py:1: DtypeWarning: Columns (2,54) have mixed types. Specify dtype option on import or set low_memory=False.
  gstore = pd.read_csv("/Users/aoluwolerotimi/Datasets/train_dejsonified.csv")


In [3]:
gstore['date'] = pd.to_datetime(gstore['date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
gstore['date'] = pd.to_datetime(gstore['date'])

In [4]:
print("Earliest cpc date")
print(gstore[gstore['medium'] == 'cpc']['date'].min())
print("Latest cpc date")
print(gstore[gstore['medium'] == 'cpc']['date'].max())

Earliest cpc date
2016-08-01 00:00:00
Latest cpc date
2017-08-01 00:00:00


CPC interactions span the whole year in the dataset, so able to sample a control group across the dataset.

In [5]:
gstore['cpcVisit'] = gstore['medium'].apply(lambda x: 1 if x == 'cpc' else 0)

In [6]:
cpc_visitors = gstore[gstore['cpcVisit'] == 1]['fullVisitorId'].unique().tolist()

In [7]:
noncpc_visitors = gstore[gstore['cpcVisit'] == 0]['fullVisitorId'].unique().tolist()

In [8]:
print(f" There are {len(cpc_visitors)} cpc vistors")
print(f" There are {len(noncpc_visitors)} non cpc vistors")

 There are 18993 cpc vistors
 There are 709414 non cpc vistors


In [9]:
# Filter gtore where 'cpcVisit' is 1
filtered = gstore[gstore['cpcVisit'] == 1]

# Further filter where 'fullVisitorId' is in the list cpc_visitor
filtered = filtered[filtered['fullVisitorId'].isin(cpc_visitors)]

# Drop duplicates to keep only the first entry for each 'fullVisitorId'
cpcdf = filtered.drop_duplicates(subset='fullVisitorId')

In [10]:
# Filter gtore where 'cpcVisit' is 0
filtered = gstore[gstore['cpcVisit'] == 0]

# Further filter where 'fullVisitorId' is in the list noncpc_visitor
filtered = filtered[filtered['fullVisitorId'].isin(noncpc_visitors)]

# Drop duplicates to keep only the first entry for each 'fullVisitorId'
noncpcdf = filtered.drop_duplicates(subset='fullVisitorId')

### **Apr 23 Session**

### Identify Potential Treatment Variables

Check out
* How much of the dataset received the treatment
* What's the spread of dates for people who received the treatment
* Any other commonalities we can use to reduce the scope from the whole dataset to a meaningfully comparative control sample?

In [3]:
gstore = pd.read_csv("/Users/aoluwolerotimi/Datasets/train_dejsonified.csv")

/var/folders/z1/1zds15w97_g7_fhs4_83dc3m0000gn/T/ipykernel_29598/3136096564.py:1: DtypeWarning: Columns (2,54) have mixed types. Specify dtype option on import or set low_memory=False.
  gstore = pd.read_csv("/Users/aoluwolerotimi/Datasets/train_dejsonified.csv")


In [4]:
gstore.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'continent', 'subContinent', 'country', 'region', 'metro', 'city',
       'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation',
       'browser', 'browserVersion', 'browserSize', 'operatingSystem',
       'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding',
       'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo',
       'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors',
       'screenResolution', 'deviceCategory', 'visits', 'hits', 'pageviews',
       'bounces', 'newVisits', 'transactionRevenue', 'campaign', 'source',
       'medium', 'keyword', 'adwordsClickInfo.criteriaParameters',
       'isTrueDirect', 'referralPath', 'adwordsClickInfo.page',
       'adwordsClickInfo.slot', 'adwordsClickInfo.gclId',
       'adwordsClickInfo.adNetworkType', 'adwordsClickInfo.isVideoAd',
       '

In [ ]:
g

In [5]:
gstore['adContent'].head(50)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
26    NaN
27    NaN
28    NaN
29    NaN
30    NaN
31    NaN
32    NaN
33    NaN
34    NaN
35    NaN
36    NaN
37    NaN
38    NaN
39    NaN
40    NaN
41    NaN
42    NaN
43    NaN
44    NaN
45    NaN
46    NaN
47    NaN
48    NaN
49    NaN
Name: adContent, dtype: object

In [7]:
gstore.shape

(903653, 55)

In [8]:
print(f"adContent sessions: {gstore['adContent'].count()}")
print(f"adContent percentage: {gstore['adContent'].count() / gstore.shape[0]}")
# adcontent_count = gstore['adContent'].count()
# print(adcontent_count)

adContent sessions: 10946
adContent percentage: 0.012113056671089455


In [9]:
adrelated = ['adwordsClickInfo.page',  'adwordsClickInfo.slot',  'adwordsClickInfo.gclId',  'adwordsClickInfo.adNetworkType',  'adContent',  'campaignCode', 'campaign']

In [10]:
for feature in adrelated:
    print(f"{feature} sessions: {gstore[feature].count()}")
    print(f"{feature} percentage: {gstore[feature].count() / gstore.shape[0]}")


adwordsClickInfo.page sessions: 21460
adwordsClickInfo.page percentage: 0.023748053733014775
adwordsClickInfo.slot sessions: 21460
adwordsClickInfo.slot percentage: 0.023748053733014775
adwordsClickInfo.gclId sessions: 21561
adwordsClickInfo.gclId percentage: 0.023859822299046204
adwordsClickInfo.adNetworkType sessions: 21460
adwordsClickInfo.adNetworkType percentage: 0.023748053733014775
adContent sessions: 10946
adContent percentage: 0.012113056671089455
campaignCode sessions: 1
campaignCode percentage: 1.1066194656577248e-06
campaign sessions: 903653
campaign percentage: 1.0


In [14]:
gstore['campaign'].head(50)


0     (not set)
1     (not set)
2     (not set)
3     (not set)
4     (not set)
5     (not set)
6     (not set)
7     (not set)
8     (not set)
9     (not set)
10    (not set)
11    (not set)
12    (not set)
13    (not set)
14    (not set)
15    (not set)
16    (not set)
17    (not set)
18    (not set)
19    (not set)
20    (not set)
21    (not set)
22    (not set)
23    (not set)
24    (not set)
25    (not set)
26    (not set)
27    (not set)
28    (not set)
29    (not set)
30    (not set)
31    (not set)
32    (not set)
33    (not set)
34    (not set)
35    (not set)
36    (not set)
37    (not set)
38    (not set)
39    (not set)
40    (not set)
41    (not set)
42    (not set)
43    (not set)
44    (not set)
45    (not set)
46    (not set)
47    (not set)
48    (not set)
49    (not set)
Name: campaign, dtype: object

In [15]:
gstore["campaign"] = gstore["campaign"].replace("(not set)", pd.NA)

In [16]:
for feature in adrelated:
    print(f"{feature} sessions: {gstore[feature].count()}")
    print(f"{feature} percentage: {gstore[feature].count() / gstore.shape[0]}")

adwordsClickInfo.page sessions: 21460
adwordsClickInfo.page percentage: 0.023748053733014775
adwordsClickInfo.slot sessions: 21460
adwordsClickInfo.slot percentage: 0.023748053733014775
adwordsClickInfo.gclId sessions: 21561
adwordsClickInfo.gclId percentage: 0.023859822299046204
adwordsClickInfo.adNetworkType sessions: 21460
adwordsClickInfo.adNetworkType percentage: 0.023748053733014775
adContent sessions: 10946
adContent percentage: 0.012113056671089455
campaignCode sessions: 1
campaignCode percentage: 1.1066194656577248e-06
campaign sessions: 38306
campaign percentage: 0.042390165251484806


**Most examples available for adwordsClickInfo.gclId sessions**

In [17]:

adwordsdf = gstore[gstore['adwordsClickInfo.gclId'].notnull()]


In [19]:
sum_transaction_revenue = adwordsdf['transactionRevenue'].sum(skipna=True)


In [22]:
adwordsdf['transactionRevenue'].count()

456

In [23]:
print(adwordsdf['transactionRevenue'].count() / adwordsdf.shape[0])

0.02114929734242382


In [24]:
adwordsdf.shape

(21561, 55)

In [21]:
adwordsdf.isna

<bound method DataFrame.isna of        channelGrouping      date        fullVisitorId  \
102        Paid Search  20160902  2870162892603227525   
271        Paid Search  20160902  9012569622109896332   
295        Paid Search  20160902  4844837653335527853   
305        Paid Search  20160902  2615187956581444986   
400        Paid Search  20160902  5407858196729335230   
...                ...       ...                  ...   
903270     Paid Search  20170104  9264990984859179657   
903283     Paid Search  20170104  1529484952694004327   
903287     Paid Search  20170104  8626798226969724640   
903295     Paid Search  20170104  4740747620184612285   
903307     Paid Search  20170104  8408834556781344877   

                             sessionId  socialEngagementType     visitId  \
102     2870162892603227525_1472858877  Not Socially Engaged  1472858877   
271     9012569622109896332_1472878281  Not Socially Engaged  1472878281   
295     4844837653335527853_1472854260  Not Socially En

In [20]:
sum_transaction_revenue

42845840000.0

In [26]:
adwordsdf['newVisits'].count()

14486

In [27]:
print(adwordsdf['newVisits'].count() / adwordsdf.shape[0])

0.6718612309262093


In [28]:
unique_visitor_ids = adwordsdf['fullVisitorId'].unique().tolist()

In [29]:
gstore['transactionRevenue'].describe()


count    1.151500e+04
mean     1.337448e+08
std      4.482852e+08
min      1.000000e+04
25%      2.493000e+07
50%      4.945000e+07
75%      1.076550e+08
max      2.312950e+10
Name: transactionRevenue, dtype: float64

In [30]:
gstore['originalRevenue'] = gstore['transactionRevenue'] / 10**6

In [31]:
gstore['originalRevenue'].describe()

count    11515.000000
mean       133.744789
std        448.285227
min          0.010000
25%         24.930000
50%         49.450000
75%        107.655000
max      23129.500000
Name: originalRevenue, dtype: float64